In [1]:
import numpy as np
import pandas as pd

In [2]:
import sys

In [4]:
sys.path.insert(0, "../src")

In [7]:
from process_SHAC import load_process_SHAC
from backdoorBERT import backdoorAdjustBERTModel

from backdoorBERT import baseModel

In [62]:
import torch

In [18]:
from NeuralModel import TransformerDataset
from torch.utils.data import DataLoader


In [40]:
df = load_process_SHAC(replaceNA="all")
label = "Drug"
txt_col = "text"
z_Categories = ["uw","mimic"]
n_zCats = len(z_Categories)

domain_col = "location"
v = 1

tmp = pd.get_dummies(pd.Categorical(df[domain_col], categories=z_Categories), prefix="confounder") * v
df = pd.concat([df,tmp], axis=1)

# df0 = df.query(f"location == '{z_Categories[0]}'").reset_index(drop=True)
# df1 = df.query(f"location == '{z_Categories[1]}'").reset_index(drop=True)

confounder_cols = ["confounder_"+x for x in z_Categories]


In [48]:
X=df['text']
y=df[['Drug']]

y_domain = df.loc[:,confounder_cols]

In [35]:
dataset = TransformerDataset(
            X=X,
            y=y,
    y_domain=y_domain,
            pretrained='bert-base-uncased',
            max_length=120,
        )

In [50]:
dataset

In [51]:
dataloader = DataLoader(dataset, shuffle=True, batch_size=50)

In [52]:
batch = next(iter(dataloader))

In [53]:
batch

{'input_ids': tensor([[  101,  2591,  2381,  ...,     0,     0,     0],
         [  101,  2591,  2381,  ...,     0,     0,     0],
         [  101,  2591,  2381,  ...,     0,     0,     0],
         ...,
         [  101, 14243,  1024,  ...,     0,     0,     0],
         [  101, 14021,  1024,  ...,     0,     0,     0],
         [  101,  2591,  2381,  ...,     0,     0,     0]]),
 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]]),
 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         ...,
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0]]),
 'labels': tensor([[1.],
         [0.],
         [0.],
         [1.],
         [0.],
         [1.],
         [0.],
         [1.],

In [54]:
model = baseModel(num_labels=2, num_zCats=2,)

In [56]:
outputs_bert = model.bert(input_ids=batch["input_ids"],
                    token_type_ids=batch["token_type_ids"],
                    attention_mask=batch["attention_mask"],)

In [58]:
outputs_pooler = outputs_bert["pooler_output"]

outputs_pooler = model.dropout(outputs_pooler)

In [60]:
outputs_pooler.shape

torch.Size([50, 768])

In [70]:
outputs_extend_confounds = torch.cat([outputs_pooler, batch['domain_labels']], 1)
outputs_extend_confounds.shape

torch.Size([50, 770])

In [71]:
outputs_main_classifier = model.main_classifier_layer(outputs_extend_confounds)

In [73]:
outputs_main_classifier

tensor([[ 0.0160,  0.5240],
        [ 0.1593,  0.5624],
        [ 0.0852,  0.7299],
        [ 0.3507,  0.5527],
        [ 0.2404,  0.5785],
        [-0.0183,  0.6881],
        [ 0.2272,  0.5761],
        [ 0.2937,  0.4312],
        [-0.0579,  0.4578],
        [ 0.0404,  0.4254],
        [-0.2518,  0.4603],
        [-0.2868,  0.4521],
        [-0.3298,  0.5922],
        [-0.0954,  0.7992],
        [ 0.1329,  0.6663],
        [-0.1032,  0.6057],
        [-0.2905,  0.5313],
        [-0.0119,  0.5222],
        [ 0.0873,  0.6932],
        [ 0.2957,  0.6708],
        [ 0.1791,  0.5760],
        [ 0.2682,  0.6526],
        [ 0.1983,  0.6499],
        [ 0.0689,  0.6755],
        [-0.3370,  0.6792],
        [-0.1984,  0.4126],
        [-0.0913,  0.4589],
        [ 0.0676,  0.6835],
        [-0.0599,  0.4434],
        [ 0.2306,  0.6996],
        [ 0.3502,  0.5303],
        [-0.1152,  0.7083],
        [ 0.0421,  0.6227],
        [-0.0108,  0.4141],
        [-0.1859,  0.4067],
        [-0.0069,  0